In [ ]:
#pip install plotly==5.8.0

In [ ]:
#!pip  install yfinance  - в колабе нужно поставить пакет

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 18.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import pandas_datareader.data as web
import yfinance as yf


In [ ]:
#временной интервал       
start ='2015-01-01'
finish = '2019-12-31' 
                

#**Датафрейм**

In [ ]:
df = yf.download("YNDX", start=start, end = finish)                            
df = df[['Open', 'High', 'Low', 'Close']]                      
df.tail(3)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2019-12-26,43.630001,43.889999,43.439999,43.730000
2019-12-27,44.099998,44.279999,43.660000,43.900002
2019-12-30,43.930000,43.970001,43.080002,43.439999


#**Основная функция**

In [ ]:
def movings2(df,money = 100000, stop_loss = 0.07, short_window = 30, long_window = 90): #аргументы функции согласно условиям
    record_date = []  #пустые списки, чтобы потом преобразовать в колонки для дф с записями
    record_sygnal = []
    record_price = []
    sig_buy = [] # пустые списки, чтобы потом сделать колонки для дальнейшей визуализации
    sig_sell =[]
    stoploss =[]
    pos = 0 #кэш, 1 - с акциями на руках
    stop_price = 100000 # задал большой стоппрайс, чтобы low было меньше до тех пор, пока не определиться цена стоплосса
                        #(после первой покупки)
    positions = [] # также пустой список для визуализации
    
    df['30_Mean'] = df['Close'].rolling(window = short_window).mean()
    df['90_Mean'] = df['Close'].rolling(window = long_window).mean()
    
    for i in range(len(df)):
        positions.append(pos)
        if (pos == 1) & (df['Low'][i] < stop_price) : # услвоие на срабатывание стоплосса
            record_date.append(df.index[i])
            record_sygnal.append('stoploss')
            record_price.append(stop_price)
            stoploss.append(stop_price)
            sig_buy.append(np.nan)
            sig_sell.append(np.nan)
            pos = 0
        elif (pos == 1) & (df.shift(-1)['Open'][i] < stop_price) : # если произошло такое, что при открытии след дня цена оказалась
                                                           # ниже стоплосса
            record_date.append(df.index[i])
            record_sygnal.append('stoploss')
            record_price.append(df['Open'][i+1])
            stoploss.append(df['Open'][i+1])
            sig_buy.append(np.nan)
            sig_sell.append(np.nan)
            pos = 0
            
        else :
            stoploss.append(np.nan)
            if (df['30_Mean'][i] > df['90_Mean'][i]) & (df.shift(1)['30_Mean'][i] < df.shift(1)['90_Mean'][i]): 
                                                                                                  # условие срабатывания
                                                                                                  # cигнала на продажу,
                                                                                                  # (условие во вторых скобках
                                                                                                  # нужно, чтобы послестоплосса
                                                                                                  # корректно работало )
                if pos == 0:
                    sig_sell.append(np.nan)
                    sig_buy.append(df['Close'][i])
                    record_date.append(df.index[i])  # cначала заносим сигнал сиг_бай, потом сам бай на след день
                    record_sygnal.append('sig_buy')
                    record_price.append(df['Close'][i]) 
                    record_date.append(df.index[i+1])
                    record_sygnal.append('buy')
                    record_price.append(df['Open'][i+1])
                    stop_price = df['Open'][i+1] * (1-stop_loss) #определение цены стоплосса
                    pos = 1
                else :
                    sig_buy.append(np.nan)
                    sig_sell.append(np.nan)
            elif df['30_Mean'][i] < df['90_Mean'][i]:  #условие срабатывания сигнала на продажу
                if pos == 1:
                    sig_buy.append(np.nan)
                    sig_sell.append(df['Close'][i]) #сначала заносим сигнал сиг_селл, потом сам селл на след день
                    record_date.append(df.index[i])
                    record_sygnal.append('sig_sell')
                    record_price.append(df['Close'][i])
                    record_date.append(df.index[i+1])
                    record_sygnal.append('sell')
                    record_price.append(df['Open'][i+1])
                    pos = 0
                else:
                    sig_buy.append(np.nan)
                    sig_sell.append(np.nan)
            else :
                sig_buy.append(np.nan)
                sig_sell.append(np.nan)
    #создаю 2 дф для заполнения таблицы с записями сигналов и подсчета прибыли
    dfpp = pd.DataFrame(list(zip(record_date,record_sygnal,record_price)),columns =['date', 'signal', 'share_price'])
    dfpp[['num_shares', 'share_value', 'rest', 'cash']]= 0 
    dfpp.iloc[0,6] = money #cash по условию
    dfpp.iloc[0,4] = np.nan
    #более читаемый код след цикла в последнем блоке(закоментированный), но он бьет ворнинги, поэтому все-таки переписал
    for i in range(1,len(dfpp)):
        if dfpp.iloc[i,1] == 'buy':
            dfpp.iloc[i,3] = round(dfpp.iloc[i-1,6] // dfpp.iloc[i,2],2)
            dfpp.iloc[i,4]= round(dfpp.iloc[i,3] * dfpp.iloc[i,2],2)
            dfpp.iloc[i,5] = dfpp.iloc[i-1,6]-  dfpp.iloc[i,4]
            dfpp.iloc[i,6] = dfpp.iloc[i,5]
        elif dfpp.iloc[i,1] == 'sig_sell':
            dfpp.iloc[i,3] = dfpp.iloc[i-1,3]
            dfpp.iloc[i,4] = np.nan
            dfpp.iloc[i,5] = dfpp.iloc[i-1,5]
            dfpp.iloc[i,6] = dfpp.iloc[i,5]
        elif dfpp.iloc[i,1] == 'sell':
            dfpp.iloc[i,3] = dfpp.iloc[i-2,3]
            dfpp.iloc[i,4]= round(dfpp.iloc[i,3] * dfpp.iloc[i,2],2)
            dfpp.iloc[i,5] = dfpp.iloc[i-2,5]
            dfpp.iloc[i,6] = dfpp.iloc[i,5] + dfpp.iloc[i,4]
        elif dfpp.iloc[i,1] == 'stoploss':
            dfpp.iloc[i,3] = dfpp.iloc[i-1,3]
            dfpp.iloc[i,4]= round(dfpp.iloc[i,3] * dfpp.iloc[i,2],2)
            dfpp.iloc[i,5] = dfpp.iloc[i-1,5]
            dfpp.iloc[i,6] = dfpp.iloc[i,5] + dfpp.iloc[i,4]
        elif dfpp.iloc[i,1] == 'sig_buy':
            dfpp.iloc[i,4] = np.nan
            dfpp.iloc[i,6] = dfpp.iloc[i-1,6]
    df['sig_sell'] = sig_sell
    df['sig_buy'] = sig_buy
    df['stoploss'] = stoploss
    df['positions'] = positions

    return dfpp,df


## **Таблица** c записями сигналов и торговыми операциями

In [ ]:
dfpp = movings2(df)[0]
dfpp



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,date,signal,share_price,num_shares,share_value,rest,cash
0,2015-11-05,sig_buy,15.740000,0,NaN,0.00,100000.00
1,2015-11-06,buy,15.550000,6430,99986.50,13.50,13.50
2,2015-12-08,stoploss,14.461500,6430,92987.45,13.50,93000.95
3,2016-04-04,sig_buy,15.060000,0,NaN,0.00,93000.95
4,2016-04-05,buy,14.910000,6237,92993.67,7.28,7.28
5,2016-09-30,sig_sell,21.049999,6237,NaN,7.28,7.28
6,2016-10-03,sell,21.360001,6237,133222.32,7.28,133229.60
7,2017-01-10,sig_buy,21.570000,0,NaN,0.00,133229.60
8,2017-01-11,buy,21.610001,6165,133225.65,3.95,3.95
9,2018-04-26,sig_sell,32.730000,6165,NaN,3.95,3.95


In [ ]:
sum = dfpp[(dfpp['signal'] == 'sell')| (dfpp['signal'] == 'stoploss')].iloc[-1,6] # cделал маску по сигналам и нашел последнюю
                                                                                  #  последнюю продажу или стоплосс
sum_change = ( sum / 100000 - 1 ) * 100 #прирост вложений в процентах
print(f"Итоговое количество денег составило {sum:.2f} единиц, прирост - {sum_change :.2f} процентов")

Итоговое количество денег составило 176444.42 единиц, прирост - 76.44 процентов


In [ ]:
df = movings2(df)[1]
df.reset_index(inplace= True) #убрал индекс в колонку, чтобы обращаться в следющем блоке к колонке дф


#**Визаулизация**

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

trace0 = go.Candlestick(x = df['Date'], open = df['Open'],
                        high = df['High'], low = df['Low'], 
                        close = df['Close'],name = 'candles')

trace1 = go.Scatter(x=df['Date'],y=df['30_Mean'],line = dict(color ='blue'), name = 'Short Moving Average' )

trace2 = go.Scatter(x=df['Date'],y=df['90_Mean'],line = dict(color ='orange'), name = 'Long Moving Average' )

trace3 = go.Scatter(x=df['Date'],y=df['sig_sell'], mode='markers',marker_symbol = 'triangle-down', marker_color='red',
                     marker_size = 13, name = 'sig_sell')

trace4 = go.Scatter(x=df['Date'],y=df['sig_buy'], mode='markers',marker_symbol = 'triangle-up',marker_color='green',
                    marker_size = 13, name = 'sig_buy')

trace5 = go.Scatter(x=df['Date'],y=df['stoploss'], mode='markers',marker_symbol = 'triangle-down', marker_color='black',
                    marker_size = 13, name = 'stoploss')
data = [trace0,trace1, trace2, trace3, trace4, trace5] #можно убрать из списка один из "трейсов" и график будет без него



fig.add_traces(data, rows = 1, cols =1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['positions'], name = 'positions'), row=2, col=1)
fig.update_layout(xaxis_rangeslider_visible = False) # убирает ползунок под графиком с свечой
fig.update_layout(xaxis_title = 'Date',yaxis_title = 'Share price')
fig.update_layout(title ='YNDX trading review visualisation')
fig.show() 